In [1]:
import numpy as np
import pandas as pd
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Select the GPU index
import scipy.io
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import math
from PIL import Image
from collections import OrderedDict
import torchvision.transforms as T
import matplotlib.pyplot as plt
import time
from torch.optim.lr_scheduler import _LRScheduler
import warnings
import spacy
from scipy.io import savemat
import dill as pickle
import thop
from torch_challenge_dataset import DeepVerseChallengeLoaderTaskTwo
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
#Parameters
onoffdict={'GPS': False, 'CAMERAS': False, 'RADAR': False}
lr=4e-3
num_epochs=200

reduction = 4
batch_size = 100
expansion=20
weight_path=f'models0/ACRNettask2/cr{reduction}/gps{onoffdict["GPS"]}_cam{onoffdict["CAMERAS"]}_rad{onoffdict["RADAR"]}/'

In [3]:
reduction

4

In [4]:
weight_path

'models0/ACRNettask2/cr4/gpsFalse_camFalse_radFalse/'

In [5]:
if not os.path.exists(weight_path):
    os.makedirs(weight_path)

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Data Analysis

In [7]:
train_dataset = DeepVerseChallengeLoaderTaskTwo(csv_path = r'./dataset_train.csv')
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True, num_workers=5)
test_dataset = DeepVerseChallengeLoaderTaskTwo(csv_path = r'./dataset_validation.csv')
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=True, num_workers=5)

In [8]:
# Get all data in numpy arrays
train_data = []
for batch in train_loader:
    train_data.append(batch[0][0].cpu().numpy())
train_data = np.concatenate(train_data, axis=0)


test_data = []
for batch in test_loader:
    test_data.append(batch[0][0].cpu().numpy())
test_data = np.concatenate(test_data, axis=0)

In [9]:
# Calculate mean and standard deviation
train_mean_real = np.mean(np.real(train_data))
train_std_real = np.std(np.real(train_data))
test_mean_real = np.mean(np.real(test_data))
test_std_real = np.std(np.real(test_data))

print(f"Train mean real: {train_mean_real}, Train std real: {train_std_real}")
print(f"Test mean real: {test_mean_real}, Test std real: {test_std_real}")

Train mean real: 1.2371337287930828e-10, Train std real: 2.50619018515863e-06
Test mean real: -3.525701286477556e-09, Test std real: 2.590656094980659e-06


In [10]:
# Calculate mean and standard deviation
train_mean_imag = np.mean(np.imag(train_data))
train_std_imag = np.std(np.imag(train_data))
test_mean_imag = np.mean(np.imag(test_data))
test_std_imag = np.std(np.imag(test_data))

print(f"Train mean imag: {train_mean_imag}, Train std imag: {train_std_imag}")
print(f"Test mean imag: {test_mean_imag}, Test std imag: {test_std_imag}")

Train mean imag: -1.0215353563047813e-10, Train std imag: 2.506833425286459e-06
Test mean imag: 3.3883891248365217e-09, Test std imag: 2.5895194539771182e-06


In [11]:
print(f"Train max real: {np.max(np.real(train_data))}, Train min real: {np.min(np.real(train_data))}")
print(f"test max real: {np.max(np.real(test_data))}, test min real: {np.min(np.real(test_data))}")

Train max real: 8.153845556080341e-05, Train min real: -7.441289199050516e-05
test max real: 8.344990783371031e-05, test min real: -7.298094715224579e-05


In [12]:
print(f"Train max imag: {np.max(np.imag(train_data))}, Train min imag: {np.min(np.imag(train_data))}")
print(f"test max imag: {np.max(np.imag(test_data))}, test min imag: {np.min(np.imag(test_data))}")

Train max imag: 7.369669765466824e-05, Train min imag: -7.740745786577463e-05
test max imag: 7.479068881366402e-05, test min imag: -7.253310468513519e-05


In [13]:
print(f"Standard deviation of absolute values: {np.std(np.abs(train_data))}")

Standard deviation of absolute values: 2.811797912727343e-06


In [14]:
train_data.shape

(11000, 1, 64, 64)

In [15]:
def CSI_reshape( y, csi_std=2.5e-06, target_std=1):
        ry = torch.real(y)
        iy= torch.imag(y)
        oy=torch.cat([ry,iy],dim=1)
        #scaling
        oy=(oy/csi_std)*target_std
        return oy

In [16]:
after_reshape = CSI_reshape( torch.from_numpy(train_data)).numpy() 

In [17]:
# Calculate mean and standard deviation

train_mean_real = np.mean(after_reshape[:,0,:,:])
train_std_real = np.std(after_reshape[:,0,:,:])
test_mean_real = np.mean(after_reshape[:,0,:,:])
test_std_real = np.std(after_reshape[:,0,:,:])

print(f"Train mean real: {train_mean_real}, Train std real: {train_std_real}")
print(f"Test mean real: {test_mean_real}, Test std real: {test_std_real}")
print(f"Train max real: {np.max(after_reshape[:,0,:,:])}, Train min real: {np.min(after_reshape[:,0,:,:])}")
print(f"test max real: {np.max(after_reshape[:,0,:,:])}, test min real: {np.min(after_reshape[:,0,:,:])}")

Train mean real: 4.948540299665183e-05, Train std real: 1.0024768114089966
Test mean real: 4.948540299665183e-05, Test std real: 1.0024768114089966
Train max real: 32.61538314819336, Train min real: -29.76515769958496
test max real: 32.61538314819336, test min real: -29.76515769958496


In [18]:
# Calculate mean and standard deviation

train_mean_imag = np.mean(after_reshape[:,1,:,:])
train_std_imag = np.std(after_reshape[:,1,:,:])
test_mean_imag = np.mean(after_reshape[:,1,:,:])
test_std_imag = np.std(after_reshape[:,1,:,:])

print(f"Train mean imag: {train_mean_imag}, Train std imag: {train_std_imag}")
print(f"Test mean imag: {test_mean_imag}, Test std imag: {test_std_imag}")
print(f"Train max imag: {np.max(after_reshape[:,1,:,:])}, Train min imag: {np.min(after_reshape[:,1,:,:])}")
print(f"test max imag: {np.max(after_reshape[:,1,:,:])}, test min imag: {np.min(after_reshape[:,1,:,:])}")

Train mean imag: -4.0861923480406404e-05, Train std imag: 1.0027332305908203
Test mean imag: -4.0861923480406404e-05, Test std imag: 1.0027332305908203
Train max imag: 29.478679656982422, Train min imag: -30.962984085083008
test max imag: 29.478679656982422, test min imag: -30.962984085083008


# Utils and Models

In [19]:
def CSI_ri_avg(y):
    ry=y[:,0,:,:]
    iy=y[:,1,:,:]
    y = torch.sqrt(ry**2+iy**2)
    return y

In [20]:
#preprocessing output as neural network doesnot understand complex numbers, without normalization
def CSI_complex2real(y):
    ry = torch.real(y)
    iy= torch.imag(y)
    oy=torch.cat([ry,iy],dim=1)
    return oy

In [21]:
def cal_model_parameters(model):
    total_param  = []
    for p1 in model.parameters():
        total_param.append(int(p1.numel()))
    return sum(total_param)

In [22]:
def NMSE_cal(x_hat, x):
    x_real = torch.reshape(torch.real(x), (x.shape[0], -1))
    x_imag = torch.reshape(torch.imag(x), (x.shape[0], -1))
    x_hat_real = torch.reshape(torch.real(x_hat), (x.shape[0], -1))
    x_hat_imag = torch.reshape(torch.imag(x_hat), (x.shape[0], -1))
    power = torch.sum(x_real** 2 + x_imag** 2, dim=1)
    mse = torch.sum((x_real - x_hat_real) ** 2 + (x_imag - x_hat_imag) ** 2, dim=1)
    nmse = torch.mean(mse / power)
    return nmse

In [23]:
class ConvBN(nn.Sequential):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, groups=1):
        if not isinstance(kernel_size, int):
            padding = [(i - 1) // 2 for i in kernel_size]
        else:
            padding = (kernel_size - 1) // 2
        super(ConvBN, self).__init__(OrderedDict([
            ('conv', nn.Conv2d(in_channels=in_planes,
                               out_channels=out_planes,
                               kernel_size=kernel_size,
                               stride=stride,
                               padding=padding,
                               groups=groups,
                               bias=False)),
            ('bn', nn.BatchNorm2d(out_planes))
        ]))

In [24]:
class ACRDecoderBlock(nn.Module):
    r""" Inverted residual with extensible width and group conv
    """
    def __init__(self, expansion):
        super(ACRDecoderBlock, self).__init__()
        width = 8 * expansion
        self.conv1_bn = ConvBN(2, width, [1, 9])
        self.prelu1 = nn.PReLU(num_parameters=width, init=0.3)
        self.conv2_bn = ConvBN(width, width, 7, groups=4 * expansion)
        self.prelu2 = nn.PReLU(num_parameters=width, init=0.3)
        self.conv3_bn = ConvBN(width, 2, [9, 1])
        self.prelu3 = nn.PReLU(num_parameters=2, init=0.3)
        self.identity = nn.Identity()

    def forward(self, x):
        identity = self.identity(x)

        residual = self.prelu1(self.conv1_bn(x))
        residual = self.prelu2(self.conv2_bn(residual))
        residual = self.conv3_bn(residual)

        return self.prelu3(identity + residual)


class ACREncoderBlock(nn.Module):
    def __init__(self):
        super(ACREncoderBlock, self).__init__()
        self.conv_bn1 = ConvBN(2, 2, [1, 9])
        self.prelu1 = nn.PReLU(num_parameters=2, init=0.3)
        self.conv_bn2 = ConvBN(2, 2, [9, 1])
        self.prelu2 = nn.PReLU(num_parameters=2, init=0.3)
        self.identity = nn.Identity()

    def forward(self, x):
        identity = self.identity(x)

        residual = self.prelu1(self.conv_bn1(x))
        residual = self.conv_bn2(residual)

        return self.prelu2(identity + residual)

In [25]:
class task2Encoder(nn.Module):
    
    def __init__(self, reduction=16, expansion=20):
        super(task2Encoder, self).__init__()
        self.total_size =8192
        n1=int(math.log2(reduction))
        self.encoder_feature = nn.Sequential(OrderedDict([
            ("conv5x5_bn", ConvBN(2, 2, 5)),
            ("prelu", nn.PReLU(num_parameters=2, init=0.3)),
            ("ACREncoderBlock1", ACREncoderBlock()),
            ("ACREncoderBlock2", ACREncoderBlock()),
        ]))
        self.encoder_fc = nn.Linear(self.total_size, self.total_size // reduction)
        self.output_sig = nn.Sigmoid()
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.Linear)):
                nn.init.xavier_uniform_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        
    def forward(self, x):
        n, c, h, w = x.detach().size()
        
        out = self.encoder_feature(x.to(torch.float32))
        out =  self.encoder_fc(out.view(n, -1))
        
        
        return out
       

In [26]:
class task2Decoder(nn.Module):
    
    def __init__(self, reduction=16, expansion=20):
        super(task2Decoder, self).__init__()
        self.total_size = 8192
        w, h =64, 64
        self.reduced_size = self.total_size//reduction
        self.decoder_fc = nn.Linear(self.total_size // reduction, self.total_size)
        self.decoder_feature = nn.Sequential(OrderedDict([
            ("conv5x5_bn", ConvBN(2, 2, 5)),
            ("prelu", nn.PReLU(num_parameters=2, init=0.3)),
            ("ACRDecoderBlock1", ACRDecoderBlock(expansion=expansion)),
            ("ACRDecoderBlock2", ACRDecoderBlock(expansion=expansion)),
            ("sigmoid", nn.Sigmoid())
        ]))
        
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.Linear)):
                nn.init.xavier_uniform_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        
        
     
    
    def forward(self, Hencoded):
        bs = Hencoded.size(0)
        #combining
        out = Hencoded.view(bs, self.reduced_size)
        # Generate final output
        out = self.decoder_fc(out)
        out = self.decoder_feature(out.view(bs, -1, 64, 64))
        
        return out

In [27]:
#complete task 2 model including encoder, decoder and channel
class task2model(nn.Module):
    def __init__(self, reduction=16, expansion=20):
        super().__init__()
        
        self.en=task2Encoder(reduction)
        
        self.de=task2Decoder(reduction,expansion)
        
    
   
    def forward(self, Hin, device, is_training): 
        
        #Encoder
        Hencoded=self.en(Hin)
        
        
        #Decoder   
        Hdecoded=self.de(Hencoded)
        

        return Hdecoded

In [28]:
class WarmUpCosineAnnealingLR(_LRScheduler):
    def __init__(self, optimizer, T_max, T_warmup, eta_min=0, last_epoch=-1):
        self.T_max = T_max
        self.T_warmup = T_warmup
        self.eta_min = eta_min
        super(WarmUpCosineAnnealingLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        if self.last_epoch < self.T_warmup:
            return [base_lr * self.last_epoch / self.T_warmup for base_lr in self.base_lrs]
        else:
            k = 1 + math.cos(math.pi * (self.last_epoch - self.T_warmup) / (self.T_max - self.T_warmup))
            return [self.eta_min + (base_lr - self.eta_min) * k / 2 for base_lr in self.base_lrs]

In [29]:
model=task2model(reduction, expansion)
print(f'Number of parameters in Task2 Encoder: {cal_model_parameters(model.en)}')
print(f'Number of parameters in Task2 Decoder: {cal_model_parameters(model.de)}')

Number of parameters in Task2 Encoder: 16779538
Number of parameters in Task2 Decoder: 16830326


In [30]:
# Training

In [31]:
#Loss

#criterion=nn.BCELoss()
#criterion = nn.CrossEntropyLoss()
criterion= nn.MSELoss().to(device)

In [32]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.98), eps=1e-9)
#optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#if SGDR == True:
#    sched = CosineWithRestarts(optimizer, T_max=n_batches)

#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, threshold=0.00001, patience=200, verbose=True)

In [33]:
scheduler = WarmUpCosineAnnealingLR(optimizer=optimizer,
                                            T_max=num_epochs * len(train_loader),
                                            T_warmup=30 * len(train_loader),
                                            eta_min=5e-5)

In [34]:
model=model.to(device)

In [35]:
start_time = time.time()
num_train_batches=len(train_loader)
num_test_batches=len(test_loader)
train_losses = []
val_losses = []

patience_counter = 0
best_val_loss = float('inf')

for i in range(num_epochs):
    loss1 = 0
    
    epoch_time = time.time()
    model.train()
    # Run the training batches
    for b, (X_train, y_train) in enumerate(train_loader):  
        optimizer.zero_grad()
        y_train=y_train.to(device)
        # Apply the model
        Xin = CSI_reshape(X_train[0])
        y_pred=model(Xin.to(device), device, is_training=True)
        y_train_reshaped=CSI_reshape(y_train)
        loss = criterion(y_pred, y_train_reshaped) 
        # Update parameters
        loss.backward()
        optimizer.step()
        scheduler.step()
        loss1=loss1+loss
        
            
    train_loss=loss1/num_train_batches  
    train_losses.append(train_loss.item())
    
    
   
    # Run the testing batches
    model.eval()
    with torch.no_grad():
        loss1=0 
        nmse1=0
        for b, (X_test, y_test) in enumerate(test_loader):
            y_test=y_test.to(device)
            # Apply the model
            Xin = CSI_reshape(X_test[0])
            y_pred=model(Xin.to(device), device, is_training=True)
            y_test_reshaped=CSI_reshape(y_test)
            loss = criterion(y_pred, y_test_reshaped) 
            loss1=loss1+loss  
            
        val_loss=loss1/num_test_batches 
        val_losses.append(val_loss.item())
    print(f'epoch:{i+1}/{num_epochs} average reshaped TL:{train_loss.item():10.8f} average reshaped VL:{val_loss.item():10.8f} epoch time:{time.time() - epoch_time:.0f} seconds, lr:{optimizer.param_groups[0]["lr"]:.2e}')               
        # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model, weight_path+"task2.pth")
        torch.save(model.en, weight_path+"task2Encoder.pth")
        torch.save(model.de, weight_path+"task2Decoder.pth")

print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed            

epoch:1/200 average reshaped TL:1.32051313 average reshaped VL:1.32134378 epoch time:398 seconds, lr:1.33e-04
epoch:2/200 average reshaped TL:1.22612083 average reshaped VL:1.25826001 epoch time:479 seconds, lr:2.67e-04
epoch:3/200 average reshaped TL:1.17824805 average reshaped VL:1.22013748 epoch time:470 seconds, lr:4.00e-04
epoch:4/200 average reshaped TL:1.13274419 average reshaped VL:1.17960775 epoch time:480 seconds, lr:5.33e-04
epoch:5/200 average reshaped TL:1.08713818 average reshaped VL:1.13226330 epoch time:436 seconds, lr:6.67e-04
epoch:6/200 average reshaped TL:1.03626120 average reshaped VL:1.07666850 epoch time:431 seconds, lr:8.00e-04
epoch:7/200 average reshaped TL:0.97578442 average reshaped VL:1.02024162 epoch time:432 seconds, lr:9.33e-04
epoch:8/200 average reshaped TL:0.91855395 average reshaped VL:0.97004163 epoch time:427 seconds, lr:1.07e-03
epoch:9/200 average reshaped TL:0.87560636 average reshaped VL:0.93586159 epoch time:432 seconds, lr:1.20e-03
epoch:10/2

epoch:75/200 average reshaped TL:0.77323842 average reshaped VL:0.91917396 epoch time:427 seconds, lr:3.36e-03
epoch:76/200 average reshaped TL:0.77246749 average reshaped VL:0.91925830 epoch time:428 seconds, lr:3.33e-03
epoch:77/200 average reshaped TL:0.77229202 average reshaped VL:0.91798568 epoch time:415 seconds, lr:3.30e-03
epoch:78/200 average reshaped TL:0.77201003 average reshaped VL:0.91838235 epoch time:425 seconds, lr:3.27e-03
epoch:79/200 average reshaped TL:0.77158314 average reshaped VL:0.91893589 epoch time:426 seconds, lr:3.24e-03
epoch:80/200 average reshaped TL:0.77101731 average reshaped VL:0.91978842 epoch time:426 seconds, lr:3.22e-03
epoch:81/200 average reshaped TL:0.77050358 average reshaped VL:0.91862458 epoch time:424 seconds, lr:3.19e-03
epoch:82/200 average reshaped TL:0.77023834 average reshaped VL:0.91992551 epoch time:430 seconds, lr:3.16e-03
epoch:83/200 average reshaped TL:0.76963001 average reshaped VL:0.91921496 epoch time:417 seconds, lr:3.13e-03
e

epoch:149/200 average reshaped TL:0.75425380 average reshaped VL:0.92371261 epoch time:463 seconds, lr:8.64e-04
epoch:150/200 average reshaped TL:0.75412083 average reshaped VL:0.92355955 epoch time:500 seconds, lr:8.35e-04
epoch:151/200 average reshaped TL:0.75397617 average reshaped VL:0.92353070 epoch time:483 seconds, lr:8.06e-04
epoch:152/200 average reshaped TL:0.75392616 average reshaped VL:0.92355222 epoch time:495 seconds, lr:7.77e-04
epoch:153/200 average reshaped TL:0.75387359 average reshaped VL:0.92340255 epoch time:496 seconds, lr:7.49e-04
epoch:154/200 average reshaped TL:0.75374484 average reshaped VL:0.92337310 epoch time:473 seconds, lr:7.22e-04
epoch:155/200 average reshaped TL:0.75360614 average reshaped VL:0.92472249 epoch time:416 seconds, lr:6.94e-04
epoch:156/200 average reshaped TL:0.75349683 average reshaped VL:0.92295629 epoch time:422 seconds, lr:6.68e-04
epoch:157/200 average reshaped TL:0.75334483 average reshaped VL:0.92457271 epoch time:441 seconds, lr:6

In [36]:
# Plotting
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [37]:
flops, params = thop.profile(model, inputs=(Xin.to(device), device, True, ), verbose=False)
print(f'Model Flops: {flops}')
print(f'Model Params Num: {params}\n')

Model Flops: 22403481600.0
Model Params Num: 33609864.0



In [38]:
np.save(weight_path+'train_loss.npy', train_losses)
np.save(weight_path+'val_loss.npy', val_losses)

# Inference

In [39]:
#Create an instance of your model
#model = task2model()

model2=torch.load(weight_path+"task2.pth").to(device)
# Run the testing batches
model2.eval()
with torch.no_grad():
    mse1=0
    for b, (X_test, y_test) in enumerate(test_loader):

        y_test=y_test.to(device)
        # Apply the model
        Xin = CSI_reshape(X_test[0])
        y_pred=model2(Xin.to(device), device, is_training=False)
        y_test_reshaped = CSI_reshape(y_test)
        mse0 = criterion(CSI_ri_avg(y_pred), CSI_ri_avg(y_test_reshaped))
        mse1+=mse0 
        
    avg_mse=mse1/num_test_batches

In [40]:
print(f'Average MSE:{avg_mse}')


Average MSE:1.5102994441986084


In [41]:
h_list = torch.tensor([])
for b, (x,h) in enumerate(test_loader):
    h = CSI_ri_avg(CSI_reshape(h))
    h_list = torch.cat([h_list,h])
target_loss = torch.mean((torch.abs(h_list) - torch.mean(torch.abs(h_list))) ** 2)

In [42]:
improvement = (target_loss.item() - avg_mse) / target_loss.item() * 100
print(f'Percentage Improvement Achieved: {improvement:.4f}%')

Percentage Improvement Achieved: -7.0030%


In [43]:
target_loss

tensor(1.4115)